In [12]:
import simpy
import random
import pandas as pd
import matplotlib

In [28]:
def patient(env,i,p_length_of_stay):
    # add 1 to bed count while the patient is in the hospital
    glob_variables.bed_count+=1
    # trigger event when the patient is better
    yield env.timeout(p_length_of_stay)
    # reduce bed count after patient leaves
    glob_variables.bed_count-=1
    
def new_patient(env,inter_arrival_time,length_of_stay):
    i = 0
    while True:
        i+=1
        # patient length of stay is drawn from exponential distribution
        p_length_of_stay = random.expovariate(1/length_of_stay)
        # create a new patient
        p = patient(env,i,p_length_of_stay)
        # start patient process
        env.process(p)
        # define next patient based on inter arrival time
        next_p = random.expovariate(1/inter_arrival_time)
        # trigger event when new patient comes
        yield env.timeout(next_p)

In [29]:
def check(env, delay):
    yield env.timeout(delay)
    while True:
        glob_variables.check_times.append(env.now)
        glob_variables.check_beds.append(glob_variables.bed_count)
        yield env.timeout(glob_variables.check_intervals)

In [30]:
env = simpy.Environment()
env.process(new_patient(env,glob_variables.mean_inter_arrival_time,glob_variables.mean_length_of_stay))
env.process(check(env,120))

<Process(check) object at 0x11c2d8cd0>

In [31]:
env.run(until=glob_variables.sim_duration)

In [ ]:
class glob_variables:
    bed_count = 0
    mean_inter_arrival_time = .2
    mean_length_of_stay = 21
    sim_duration = 548
    check_times = []
    check_beds = []
    check_intervals = 7

In [32]:
glob_variables.check_beds

[108,
 106,
 97,
 100,
 98,
 100,
 93,
 101,
 98,
 105,
 114,
 112,
 102,
 90,
 96,
 95,
 106,
 111,
 109,
 94,
 91,
 86,
 100,
 97,
 91,
 107,
 102,
 99,
 101,
 112,
 122,
 114,
 107,
 112,
 100,
 121,
 122,
 126,
 103,
 111,
 110,
 110,
 112,
 109,
 99,
 104,
 105,
 100,
 102,
 99,
 103,
 97,
 80,
 90,
 98,
 102,
 105,
 115,
 111,
 117,
 116,
 111]